In [2]:
pip install pandas 

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade pip #upgrade pip 

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd #import module 

In [5]:
pip install pyspark #install pysparks 

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pyspark findspark #use findspark 

Note: you may need to restart the kernel to use updated packages.


In [7]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,year #start pysparks session 

In [8]:
spark = SparkSession.builder.appName("Job Count").getOrCreate()  #build session

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/05 12:11:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
df = spark.read.csv("postings.csv", header = True) #read csv 

In [10]:
df.show() #show dataframe 

24/10/05 12:11:37 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+--------------------+----------+----------+---------+----------+---------------+----------+--------------------+-------------------+--------------------+--------------------+------------------+---------------+---------------+----------------+--------------------+---------------+--------------------------+-----------+-----------+--------------+-----------+---------+--------+-----------------+-----------------+--------+----+
|              job_id|        company_name|               title|         description|max_salary|pay_period| location|company_id|          views|med_salary|          min_salary|formatted_work_type|             applies|original_listed_time|    remote_allowed|job_posting_url|application_url|application_type|              expiry|    closed_time|formatted_experience_level|skills_desc|listed_time|posting_domain|  sponsored|work_type|currency|compensation_type|normalized_salary|zip_code|fips|
+-----------------

In [11]:
internships_df = df.filter(df['description'].rlike("(?i)intern")) #check for intern in description

In [12]:
internships_df.show() #look for searches 

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+---------------+--------------------+-------------------+-------+--------------------+--------------+--------------------+---------------+-----------------+---------------+-----------+--------------------------+--------------------+---------------+--------------+---------+---------+--------+-----------------+-----------------+--------+-----+
|              job_id|        company_name|               title|         description|          max_salary|          pay_period|            location|          company_id|    views|     med_salary|          min_salary|formatted_work_type|applies|original_listed_time|remote_allowed|     job_posting_url|application_url| application_type|         expiry|closed_time|formatted_experience_level|         skills_desc|    listed_time|posting_domain|sponsored|work_type|currency|compen

In [13]:
city_internship_counts = internships_df.groupBy('location').count() #look @ loaction 

In [14]:
city_internship_counts_sorted = city_internship_counts.orderBy(col('count').desc()) #count by columns 

In [15]:
city_internship_counts_sorted.show() 

+--------------------+-----+
|            location|count|
+--------------------+-----+
|                NULL| 6275|
|       United States|  153|
|        New York, NY|   43|
| Anesthesiology a...|   33|
| be asked to code...|   29|
|New York City Met...|   28|
|    offer assistance|   28|
|         Chicago, IL|   25|
|                 age|   23|
|         Houston, TX|   21|
|         Atlanta, GA|   19|
|      Washington, DC|   18|
| including spam/j...|   18|
|New York, United ...|   17|
|          Dallas, TX|   17|
|          Boston, MA|   16|
| and Israel and t...|   16|
|               Excel|   13|
|         Phoenix, AZ|   13|
| the Company and ...|   13|
+--------------------+-----+
only showing top 20 rows



In [16]:
from pyspark.sql.functions import trim
from pyspark.sql.functions import split

In [17]:
city_internship_counts_sorted = city_internship_counts_sorted.withColumn('location_clean', trim(col('location')))
 #trim by commas

In [18]:
city_internship_counts_sorted.show()

+--------------------+-----+--------------------+
|            location|count|      location_clean|
+--------------------+-----+--------------------+
|                NULL| 6275|                NULL|
|       United States|  153|       United States|
|        New York, NY|   43|        New York, NY|
| Anesthesiology a...|   33|Anesthesiology an...|
| be asked to code...|   29|be asked to code ...|
|New York City Met...|   28|New York City Met...|
|    offer assistance|   28|    offer assistance|
|         Chicago, IL|   25|         Chicago, IL|
|                 age|   23|                 age|
|         Houston, TX|   21|         Houston, TX|
|         Atlanta, GA|   19|         Atlanta, GA|
|      Washington, DC|   18|      Washington, DC|
| including spam/j...|   18|including spam/ju...|
|New York, United ...|   17|New York, United ...|
|          Dallas, TX|   17|          Dallas, TX|
|          Boston, MA|   16|          Boston, MA|
| and Israel and t...|   16|and Israel and tr...|


In [19]:
df_with_comma = city_internship_counts_sorted.filter(col('location').rlike(",")) #harmonize 


In [20]:
df_with_comma.show()

+--------------------+-----+--------------------+
|            location|count|      location_clean|
+--------------------+-----+--------------------+
|        New York, NY|   43|        New York, NY|
|         Chicago, IL|   25|         Chicago, IL|
|         Houston, TX|   21|         Houston, TX|
|         Atlanta, GA|   19|         Atlanta, GA|
|      Washington, DC|   18|      Washington, DC|
|          Dallas, TX|   17|          Dallas, TX|
|New York, United ...|   17|New York, United ...|
|          Boston, MA|   16|          Boston, MA|
|         Phoenix, AZ|   13|         Phoenix, AZ|
|       Charlotte, NC|   12|       Charlotte, NC|
|      Greenville, SC|   11|      Greenville, SC|
|Texas, United States|   11|Texas, United States|
|       San Diego, CA|   11|       San Diego, CA|
|          Austin, TX|   11|          Austin, TX|
|          Denver, CO|   10|          Denver, CO|
|         Raleigh, NC|   10|         Raleigh, NC|
|           Greer, SC|   10|           Greer, SC|


In [21]:
df_with_comma = df_with_comma.drop("location")

In [22]:
df_with_comma = df_with_comma.select("location_clean", "count") #clean locations

In [23]:
df_renamed = df_with_comma.withColumnRenamed("location_clean", "location")

In [24]:
df_renamed.show() #show datas 

+--------------------+-----+
|            location|count|
+--------------------+-----+
|        New York, NY|   43|
|         Chicago, IL|   25|
|         Houston, TX|   21|
|         Atlanta, GA|   19|
|      Washington, DC|   18|
|          Dallas, TX|   17|
|New York, United ...|   17|
|          Boston, MA|   16|
|         Phoenix, AZ|   13|
|       Charlotte, NC|   12|
|      Greenville, SC|   11|
|Texas, United States|   11|
|       San Diego, CA|   11|
|          Austin, TX|   11|
|          Denver, CO|   10|
|         Raleigh, NC|   10|
|           Greer, SC|   10|
|           Tampa, FL|    9|
|         Seattle, WA|    9|
|    Indianapolis, IN|    8|
+--------------------+-----+
only showing top 20 rows



In [25]:
pandas_df = df_renamed.toPandas() #print pandas 

In [26]:
df_filtered = pandas_df[~pandas_df['location'].isin(["New York, United States", "Texas, United States", "California, United States"])]

In [27]:
print(df_filtered) #filtered data 

                               location  count
0                          New York, NY     43
1                           Chicago, IL     25
2                           Houston, TX     21
3                           Atlanta, GA     19
4                        Washington, DC     18
..                                  ...    ...
496                           Ocala, FL      1
497                            York, PA      1
498  Albany, New York Metropolitan Area      1
499                  Oakland County, MI      1
500                          Newark, DE      1

[498 rows x 2 columns]


In [28]:
locations_list = df_filtered['location'].to_list() #look @ list 

In [29]:
my_dict = df_filtered.set_index('location')['count'].to_dict()

In [30]:
for key in my_dict.keys():
    if ", TN" in key:
        print(key,my_dict[key])

Nashville, TN 3
Chattanooga, TN 3
Columbia, TN 2
Kingsport, TN 1
Gallatin, TN 1
Memphis, TN 1
Franklin, TN 1
Johnson City, TN 1


In [31]:
df_first_20 = df_filtered.head(20)

In [32]:
list_of_locations = df_first_20["location"].to_list()

In [33]:
print(list_of_locations)

['New York, NY', 'Chicago, IL', 'Houston, TX', 'Atlanta, GA', 'Washington, DC', 'Dallas, TX', 'Boston, MA', 'Phoenix, AZ', 'Charlotte, NC', 'San Diego, CA', 'Greenville, SC', 'Austin, TX', 'Denver, CO', 'Raleigh, NC', 'Greer, SC', 'Seattle, WA', 'Tampa, FL', 'Indianapolis, IN', 'Orlando, FL', 'Cincinnati, OH']


In [34]:
counties = [
    "New York County",  # New York, NY
    "Cook County",  # Chicago, IL
    "Harris County",  # Houston, TX
    "Fulton County", # Atlanta, GA
    "Independent city",  # Washington, DC
    "Dallas County",  # Dallas, TX
    "Suffolk County",  # Boston, MA
    "Maricopa County",  # Phoenix, AZ
    "Mecklenburg County",  # Charlotte, NC
    "San Diego County",  # San Diego, CA
    "Greenville County",  # Greenville, SC
    "Travis County",  # Austin, TX
    "Denver County",  # Denver, CO
    "Wake County",  # Raleigh, NC
    "Greenville County",  # Greer, SC
    "King County",  # Seattle, WA
    "Hillsborough County",  # Tampa, FL
    "Marion County",  # Indianapolis, IN
    "Orange County",  # Orlando, FL
    "Hamilton County"  # Cincinnati, OH
]


In [36]:
df_first_20.loc[:, 'County'] = counties

In [45]:
df_first_20 = df_first_20[['location', 'County', 'count']]

In [56]:
df_first_20.rename(columns={'count': 'Internship Postings'}) #get the postings

,location,County,Internship Postings
0,"New York, NY",New York County,43
1,"Chicago, IL",Cook County,25
2,"Houston, TX",Harris County,21
3,"Atlanta, GA",Fulton County,19
4,"Washington, DC",Independent city,18
5,"Dallas, TX",Dallas County,17
7,"Boston, MA",Suffolk County,16
8,"Phoenix, AZ",Maricopa County,13
9,"Charlotte, NC",Mecklenburg County,12
11,"San Diego, CA",San Diego County,11


In [57]:
print(df_first_20) #print the counties 

            location               County  count
0       New York, NY      New York County     43
1        Chicago, IL          Cook County     25
2        Houston, TX        Harris County     21
3        Atlanta, GA        Fulton County     19
4     Washington, DC     Independent city     18
5         Dallas, TX        Dallas County     17
7         Boston, MA       Suffolk County     16
8        Phoenix, AZ      Maricopa County     13
9      Charlotte, NC   Mecklenburg County     12
11     San Diego, CA     San Diego County     11
12    Greenville, SC    Greenville County     11
13        Austin, TX        Travis County     11
14        Denver, CO        Denver County     10
15       Raleigh, NC          Wake County     10
16         Greer, SC    Greenville County     10
17       Seattle, WA          King County      9
18         Tampa, FL  Hillsborough County      9
19  Indianapolis, IN        Marion County      8
21       Orlando, FL        Orange County      8
22    Cincinnati, OH

In [48]:
new_series = df_first_20["County"]

In [49]:
new_df = new_series.to_frame()

In [50]:
print(new_df)

                 County
0       New York County
1           Cook County
2         Harris County
3         Fulton County
4      Independent city
5         Dallas County
7        Suffolk County
8       Maricopa County
9    Mecklenburg County
11     San Diego County
12    Greenville County
13        Travis County
14        Denver County
15          Wake County
16    Greenville County
17          King County
18  Hillsborough County
19        Marion County
21        Orange County
22      Hamilton County


In [1]:
internship_postings = df_first_20["count"]

NameError: name 'df_first_20' is not defined

In [59]:
new_df["Internship Postings by County"] = internship_postings 

In [60]:
print(new_df)

                 County  Internship Postings by County
0       New York County                             43
1           Cook County                             25
2         Harris County                             21
3         Fulton County                             19
4      Independent city                             18
5         Dallas County                             17
7        Suffolk County                             16
8       Maricopa County                             13
9    Mecklenburg County                             12
11     San Diego County                             11
12    Greenville County                             11
13        Travis County                             11
14        Denver County                             10
15          Wake County                             10
16    Greenville County                             10
17          King County                              9
18  Hillsborough County                              9
19        

In [54]:
new_df.to_csv("internships_by_county.csv", index = False)

                 County
0       New York County
1           Cook County
2         Harris County
3         Fulton County
4      Independent city
5         Dallas County
7        Suffolk County
8       Maricopa County
9    Mecklenburg County
11     San Diego County
12    Greenville County
13        Travis County
14        Denver County
15          Wake County
16    Greenville County
17          King County
18  Hillsborough County
19        Marion County
21        Orange County
22      Hamilton County


In [63]:
pip install numpy<2

/bin/bash: line 1: 2: No such file or directory
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [12]:
import numpy as np

In [16]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.
